# **Clustering Top 5 football players**

### Les imports

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv
import os
import time
import re

### Récuparation de tous les liens des équipes du top 5 européen

In [19]:
class TeamScraper:
    def __init__(self, url):
        self.url = url

    def scrape_teams(self):
        response = requests.get(self.url)
        soup = BeautifulSoup(response.content, 'html.parser')
        team_table = soup.find('table', {'id': 'big5_table'})
        teams_data = []

        for row in team_table.find('tbody').find_all('tr'):
            team_cell = row.find('td', {'data-stat': 'team'})
            if team_cell:
                team_name = team_cell.get_text(strip=True)
                team_link = 'https://fbref.com' + team_cell.find('a')['href'] + '/'
                teams_data.append([team_name, team_link])

        # Save team data to CSV
        with open('../data/teams_data.csv', mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Team Name', 'Team URL'])
            writer.writerows(teams_data)

        print("Teams data saved to teams_data.csv")

In [20]:
team_scraper = TeamScraper('https://fbref.com/en/comps/Big5/Big-5-European-Leagues-Stats')
team_scraper.scrape_teams()

Teams data saved to teams_data.csv


### Récuparation de tous les liens des joueurs de chaque équipe

In [57]:
class PlayerScraper:
    def __init__(self, teams_file):
        self.teams_file = teams_file

    def scrape_players(self):
        # Open the output CSV file in append mode (use mode='a')
        output_file = '../data/players_data.csv'
        
        # Open CSV file once outside the player loop
        with open(output_file, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Player Name', 'Position', 'Player URL'])  # Write header once

            # Open the teams file and iterate through teams
            with open(self.teams_file, newline='') as csvfile:
                reader = csv.reader(csvfile)
                next(reader)  # Skip header row
                
                # Loop through each team
                for row in reader:
                    team_name, team_url = row
                    print(f"Scraping players from {team_name}...")

                    # Fetch the page content and parse it
                    response = requests.get(team_url)
                    soup = BeautifulSoup(response.content, 'html.parser')

                    # Find the player table
                    player_table = soup.find('table', {'class': 'stats_table sortable min_width'})

                    # Iterate through player rows
                    for player_row in player_table.find('tbody').find_all('tr'):

                        # Extract player name from the <th> element
                        player_name = player_row.find('th', {'data-stat': 'player'}).get_text(strip=True)

                        # Extract player position from the <td> element
                        player_position = player_row.find('td', {'data-stat': 'position'}).get_text(strip=True)

                        # Extract the player link from the <a> inside the <th> element
                        player_link = player_row.find('th', {'data-stat': 'player'}).find('a')['href']
                        player_link = 'https://fbref.com' + player_link
                        print(player_name, " - ", player_position, "- ", player_link)

                        # Write player data to the CSV file
                        writer.writerow([player_name, player_position, player_link])

                    # Add a delay after each request to avoid being blocked (5 requests per minute = 12 seconds delay)
                    time.sleep(12)

        print(f"Players data saved to {output_file}")

In [58]:
player_scraper = PlayerScraper('../data/teams_data.csv')
player_scraper.scrape_players()

Scraping players from Monaco...
Philipp Köhn  -  GK -  https://fbref.com/en/players/ff74841d/Philipp-Kohn
Denis Zakaria  -  MF -  https://fbref.com/en/players/384d58d9/Denis-Zakaria
Eliesse Ben Seghir  -  MF,FW -  https://fbref.com/en/players/3bd65247/Eliesse-Ben-Seghir
Vanderson  -  DF -  https://fbref.com/en/players/ee45fd85/Vanderson
Thilo Kehrer  -  DF -  https://fbref.com/en/players/51dbeea9/Thilo-Kehrer
Takumi Minamino  -  MF -  https://fbref.com/en/players/f833a830/Takumi-Minamino
Lamine Camara  -  MF -  https://fbref.com/en/players/19c2ffa4/Lamine-Camara
Folarin Balogun  -  FW -  https://fbref.com/en/players/31822f8c/Folarin-Balogun
Maghnes Akliouche  -  FW,MF -  https://fbref.com/en/players/b625b241/Maghnes-Akliouche
Mohammed Salisu  -  DF -  https://fbref.com/en/players/0b33f6ad/Mohammed-Salisu
Wilfried Singo  -  DF -  https://fbref.com/en/players/8b609c34/Wilfried-Singo
Breel Embolo  -  FW -  https://fbref.com/en/players/0b4f388a/Breel-Embolo
Caio Henrique Oliveira Silva  - 

### Récuparation des scouting report de tous les joueurs

In [63]:
class DataPlayerExtractor:
    def __init__(self, players_data_path, output_csv_path='../data/scouting_report.csv'):
        self.players_data_path = players_data_path
        self.output_csv_path = output_csv_path
        self.soup = None
        self.players_df = None

    def fetch_data(self, url):
        """Fetch the content of the webpage."""
        try:
            response = requests.get(url)
            response.raise_for_status()
            self.soup = BeautifulSoup(response.text, 'html.parser')
        except requests.exceptions.HTTPError as http_err:
            print(f"HTTP error occurred: {http_err}")
        except Exception as err:
            print(f"Other error occurred: {err}")
    
    def extract_player_name(self):
        """Extract the player name from the webpage."""
        if self.soup:
            player_name_tag = self.soup.find('h1')
            return player_name_tag.text.strip() if player_name_tag else "Nom non trouvé"
        return None

    def extract_scouting_report(self):
        """Extract the scouting report (if any) from the webpage."""
        if self.soup:
            table = self.soup.find('table')
            if not table:
                print("Table introuvable.")
                return None, None
            
            headers = []
            rows_data = []
            
            # Extraction des en-têtes de tableau
            thead = table.find('thead')
            if thead:
                headers = [th.text.strip() for th in thead.find_all('th')]

            # Extraction des lignes du tableau
            tbody = table.find('tbody')
            if tbody:
                for row in tbody.find_all('tr'):
                    row_data = []
                    for cell in row.find_all(['th', 'td']):
                        row_data.append(cell.text.strip())
                    
                    # Ignorer les lignes vides
                    if any(row_data):
                        rows_data.append(row_data)
            
            return headers, rows_data
        return None, None

    def save_to_csv(self, player_name, rows_data):
        """Save player data to a CSV file."""
        # Colonnes à utiliser dans le CSV
        columns = ['player_name'] + [row[0] for row in rows_data]

        # Extraire uniquement les valeurs (sans le nom des statistiques)
        values = [player_name] + [row[1] for row in rows_data]

        # Vérifier si le fichier CSV existe déjà
        file_exists = os.path.isfile(self.output_csv_path)

        # Créer le répertoire 'data' si nécessaire
        os.makedirs(os.path.dirname(self.output_csv_path), exist_ok=True)

        # Ajouter les données au CSV
        with open(self.output_csv_path, 'a', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            if not file_exists:
                # Écrire les en-têtes si le fichier est créé pour la première fois
                writer.writerow(columns)
            # Écrire les données du joueur
            writer.writerow(values)
        
        print(f"Données sauvegardées dans {self.output_csv_path}")

    def process_players(self):
        """Process all players listed in the input CSV."""
        # Charger le fichier players_data.csv
        self.players_df = pd.read_csv(self.players_data_path)

        # Initialiser le fichier de sortie scouting_report.csv s'il n'existe pas
        if not os.path.isfile(self.output_csv_path):
            with open(self.output_csv_path, 'w', newline='', encoding='utf-8') as f:
                writer = csv.writer(f)
                writer.writerow(['Player Name', 'Stat Name 1', 'Stat Value 1', 'Stat Name 2', 'Stat Value 2', '...'])

        # Boucle sur chaque joueur et extraction des données
        for index, row in self.players_df.iterrows():
            player_name = row['Player Name']
            player_url = row['Player URL']
            print(f"Processing {player_name} ({player_url})")

            self.fetch_data(player_url)
            headers, scouting_data = self.extract_scouting_report()
            
            if scouting_data:
                self.save_to_csv(player_name, scouting_data)
            else:
                print(f"No scouting report found for {player_name}.")
            
            # Attendre 12 secondes avant de traiter le prochain joueur
            time.sleep(12)

In [65]:
# Utilisation de la classe pour scraper les données des joueurs et les enregistrer dans un fichier CSV
players_data_path = '../data/players_data.csv'  # Chemin vers le fichier des joueurs
output_csv_path = '../data/scouting_report.csv'  # Chemin vers le fichier CSV de sortie

extractor = DataPlayerExtractor(players_data_path, output_csv_path)
extractor.process_players()

Processing Philipp Köhn (https://fbref.com/en/players/ff74841d/Philipp-Kohn)
Données sauvegardées dans ../data/scouting_report.csv
Processing Denis Zakaria (https://fbref.com/en/players/384d58d9/Denis-Zakaria)
Données sauvegardées dans ../data/scouting_report.csv
Processing Eliesse Ben Seghir (https://fbref.com/en/players/3bd65247/Eliesse-Ben-Seghir)
Données sauvegardées dans ../data/scouting_report.csv
Processing Vanderson (https://fbref.com/en/players/ee45fd85/Vanderson)
Données sauvegardées dans ../data/scouting_report.csv
Processing Thilo Kehrer (https://fbref.com/en/players/51dbeea9/Thilo-Kehrer)
Données sauvegardées dans ../data/scouting_report.csv
Processing Takumi Minamino (https://fbref.com/en/players/f833a830/Takumi-Minamino)
Données sauvegardées dans ../data/scouting_report.csv
Processing Lamine Camara (https://fbref.com/en/players/19c2ffa4/Lamine-Camara)
Données sauvegardées dans ../data/scouting_report.csv
Processing Folarin Balogun (https://fbref.com/en/players/31822f8c/F

### Nettoyage des données 

In [8]:
class ScoutingReportCleaner:
    def __init__(self, file_path, players_data_path):
        """Initialise la classe avec le chemin du fichier CSV et le fichier des joueurs."""
        self.file_path = file_path
        self.players_data_path = players_data_path
    
    def clean_and_save_data(self, output_path):
        """Charge, nettoie les données, ajoute la position des joueurs et sauvegarde les résultats dans un fichier CSV."""
        # Charger le fichier CSV du scouting report dans un DataFrame
        data = pd.read_csv(self.file_path)
        
        # Charger le fichier players_data.csv contenant le nom et la position des joueurs
        players_data = pd.read_csv(self.players_data_path)
        
        # Supprimer les doublons dans players_data basés sur 'Player Name' pour garder seulement la première occurrence
        players_data.drop_duplicates(subset=['Player Name'], inplace=True)
        
        # Renommer la colonne "Player Name" pour correspondre à "player_name" dans le rapport de scouting
        players_data.rename(columns={'Player Name': 'player_name'}, inplace=True)
        
        # Fonction interne pour vérifier si une valeur est un nombre valide ou un pourcentage
        def is_valid_value(val):
            # Vérifier si c'est un nombre avec ou sans pourcentage (ex: "86.3%", "-2.8")
            return bool(re.match(r'^-?\d+(\.\d+)?%?$', str(val)))
        
        # Fonction interne pour vérifier si une ligne est valide (chaque colonne après le nom doit être un nombre ou un pourcentage)
        def row_is_valid(row):
            return all(is_valid_value(val) for val in row[1:])
        
        # Nettoyer les données en conservant uniquement les lignes valides
        cleaned_data = data[data.apply(row_is_valid, axis=1)]
        
        # Fusionner sur la colonne "player_name" pour ajouter la colonne Position
        merged_data = pd.merge(cleaned_data, players_data[['player_name', 'Position']], on='player_name', how='left')
        
        # Supprimer les doublons sur toutes les colonnes
        merged_data.drop_duplicates(inplace=True)
        
        # Réorganiser les colonnes pour placer "Position" juste après "player_name"
        columns = ['player_name', 'Position'] + [col for col in cleaned_data.columns if col != 'player_name']
        final_data = merged_data[columns]
        
        # Sauvegarder les données nettoyées et enrichies dans un nouveau fichier CSV
        final_data.to_csv(output_path, index=False, header=True)
        
        print(f"Données nettoyées et enrichies sauvegardées dans {output_path}.")
        return final_data


In [9]:
# Utilisation de la classe
file_path = '../data/scouting_report.csv'  # Remplacez par votre chemin de fichier
players_data_path = '../data/players_data.csv'  # Chemin vers le fichier players_data.csv
output_path = '../data/cleaned_scouting_report.csv'  # Chemin de sortie pour les données nettoyées

# Créer une instance de la classe et nettoyer + enrichir les données
cleaner = ScoutingReportCleaner(file_path, players_data_path)

# Nettoyer et sauvegarder les données nettoyées
cleaned_data = cleaner.clean_and_save_data(output_path)

# Afficher un aperçu des données nettoyées
print("Aperçu des données nettoyées et enrichies :")
print(cleaned_data.head())

Données nettoyées et enrichies sauvegardées dans ../data/cleaned_scouting_report.csv.
Aperçu des données nettoyées et enrichies :
          player_name Position Buts (sans les pénaltys)  \
0       Denis Zakaria       MF                     0.22   
1  Eliesse Ben Seghir    MF,FW                     0.37   
2           Vanderson       DF                     0.16   
3        Thilo Kehrer       DF                     0.13   
4     Takumi Minamino       MF                     0.29   

  npxG: xG sans les pénaltys Total des tirs Passes décisives  \
0                       0.10           1.11             0.04   
1                       0.17           3.22             0.28   
2                       0.08           1.24             0.16   
3                       0.05           0.74             0.04   
4                       0.28           2.45             0.16   

  xAG: Prévu(s) Buts assistés npxG + xAG Actions menant à un tir  \
0                        0.05       0.15                    1.